 <div style="text-align:center;">
   <span style="color:green; font-size:2em; font-weight:bold;">Exploratory Data Analysis (EDA)</span><br><br>
</div>


# <span style="color:green; text-align:center;">Introduction</span>

Dans cette partie, nous nous intéressons à la visualisation des données ainsi qu'à l'analyse descriptive du jeu de données **"final_data"** obtenu suite au nettoyage des bases de données initiales du fichier "Préparation_données.ipynb". <br>

- Pour étudier les tendances moyennes, nous allons se concentrer uniquement sur les observations de la période s'étallant de **2019** à **2023**. Nous voulons à partir de cela visualiser les niveaux moyens de chomage et de croissance économique pour les cinq dernières années. <br>

- Pour les autres représentations, la période considérée est celle s'étallant de **1994** à **2023**. <br>

Pour cela, quatres applications ont été créées.

In [ ]:
# Import des packages
from importlib import reload
import declarations as d
reload(d)

In [ ]:
# Importation du jeu de données
df = d.pd.read_csv("./bases/final_data.csv")

In [ ]:
# Visualisation 
df.head()

## <span style="color:green; text-align:center;">I- Présentation de la base</span>

In [ ]:
# Dimensions de la DF
df.shape

In [ ]:
# Informations sur la DF (nombre de valeurs non nulles, type de données de chaque colonne...)
df.info()

In [ ]:
# Générer des statistiques descriptives récapitulatives d'une DataFrame
df.describe()

In [ ]:
# Telécharger le shapefile
world = d.gpd.read_file(d.gpd.datasets.get_path('naturalearth_lowres'))

# Transformer les coordonnées géographiques en coordonnées projetées
world.to_crs('EPSG:4326') 

# Renommer la colonne iso_a3
world=world.rename(columns={'iso_a3': 'COUNTRY'})

In [ ]:
# Visualisation
world.head()

In [ ]:
# Sélectionner certaines colonnes
columns_to_keep = ['geometry', 'name','COUNTRY', 'continent']
world = world[columns_to_keep]

# Fusionner les deux DataFrames
world_df = d.pd.merge(world, df, how='inner', on=['COUNTRY'])

In [ ]:
# Visualisation
world_df.head()

## <span style="color:green; text-align:center;">II- Visualisation des données </span>

In [ ]:
# Plot
world_df.plot(color='lightgrey')
d.plt.show()

## <span style="color:green; text-align:center;">II-1. APPLICATION 1</span>

La première application consiste à créer une application web interactive affichant une carte choroplèthe basée sur les données fournies dans le DataFrame world_df (shapefile).

In [ ]:
# Créer un Dash app
app1 = d.dash.Dash(__name__)

# Définir le layout 
app1.layout = d.html.Div([
    # Sélecteur d'année
    d.dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in sorted(world_df['YEAR'].unique())],
        value=world_df['YEAR'].min(),  #valeur par defaut de l'année
        multi=False,
    ),
    # Sélecteur de variables
    d.dcc.Dropdown(
        id='variable-dropdown',
        options=[
            {'label': 'Unemployment Rate', 'value': 'Unemployment_rate'},
            {'label': 'GDP growth Rate', 'value': 'GDP_rate'},
            {'label': 'Life expentancy', 'value': 'life_expentancy'},
            {'label': 'Population growth rate', 'value': 'pop_growth_rate'}
          
        ],
        value='Unemployment_rate',  # valeur par défaut de la variable representée
        multi=False,
    ),
    d.dcc.Graph(id='choropleth-map'),
])

# Définir un callback  pour mettre à jour la carte sur l'année selectionnée 
@app1.callback(
    d.Output('choropleth-map', 'figure'),
    [d.Input('year-dropdown', 'value'),
     d.Input('variable-dropdown', 'value')]
)
def update_map(selected_year, selected_variable):
    subset_gdf = world_df[world_df['YEAR'] == selected_year]
    subset_gdf.index = subset_gdf['name']
    fig = d.px.choropleth(
        subset_gdf,
        geojson=subset_gdf.geometry,
        locations=subset_gdf.index,
        color=selected_variable,
        projection="natural earth",
        title=f'{selected_variable} in {selected_year}',
    )
    fig.update_geos(fitbounds="locations", visible=True)
  
    fig.update_layout ( margin = { "r" : 0 , "t" : 0 , "l" : 0 , "b" : 0 }) 
    
    return fig

# Exécuter l'application
if __name__ == '__main__':
    app1.run_server(debug=True, port=8050)

## <span style="color:green; text-align:center;">II-2 APPLICATION 2 : Visualisation des séries temporelles par contient</span>

La deuxième application consiste à créer une application web interactive affichant les plots des séries temporelles des différents indicateurs pour différents pays.

In [ ]:
# Créer un Dash app
app2 = d.dash.Dash(__name__)

# Créer le graphique initial
fig = d.px.line(world_df, x='YEAR', y=["Unemployment_rate"], color='name')

# Créer la mise en page de votre application Dash
app2.layout = d.html.Div([
    # Sélecteur de continents
    d.dcc.Dropdown(
        id='continent-selector',
        options=[
            {'label': continent, 'value': continent}
            for continent in world_df['continent'].unique()
        ],
        multi=True,
        value=world_df['continent'].unique()  # Sélectionner tous les continents par défaut
    ),
    # Sélecteur de variables
    d.dcc.Dropdown(
        id='variable-dropdown',
        options=[
            {'label': 'Unemployment Rate', 'value': 'Unemployment_rate'},
            {'label': 'GDP growth Rate', 'value': 'GDP_rate'},
            {'label': 'Life expentancy', 'value': 'life_expentancy'},
            {'label': 'Population growth rate', 'value': 'pop_growth_rate'}
          
        ],
        value='Unemployment_rate',  # Valeur par defaut de la variable representée
        multi=False,
    ),
    
    # Graphique
    d.dcc.Graph(
        id='line-chart',
        figure=fig
    )
])

# Définir la logique de la mise à jour du graphique en fonction de la sélection du sélecteur
@app2.callback(
    d.Output('line-chart', 'figure'),
    [d.Input('variable-dropdown', 'value'),
     d.Input('continent-selector', 'value')]
)
def update_graph2(selected_variable, selected_continents):
    filtered_df = world_df[world_df['continent'].isin(selected_continents)]
    fig = d.px.line(filtered_df, x='YEAR', y=[selected_variable], color='name',
                   labels={'YEAR': 'Année', selected_variable : f"{selected_variable}"})
    
    
    return fig


# Exécuter l'application Dash
if __name__ == '__main__':
    app2.run_server(debug=True, port=8051)

## <span style="color:green; text-align:center;">II-3 APPLICATION 3 : Visualisation de la tendance moyenne </span>

La troisième application consiste à créer une application web interactive affichant les barplots (diagramme à barres) des valeurs moyennes des différents indicateurs pour les différents pays.

In [ ]:
# Tronquer la DataFrame pour garder uniquement les lignes à partir de 2019
df_tronque = world_df.loc[world_df.YEAR >= '2019']

# Sélectionner uniquement les 4 colonnes 'Unemployment_rate', 'GDP_rate', 'life_expentancy' et 'pop_growth_rate', puis faire la moyenne dans chaque groupe
df_group = df_tronque.groupby('name')[
    ['Unemployment_rate', 'GDP_rate', 'life_expentancy', 'pop_growth_rate']
    ].mean().reset_index()

In [ ]:
# Visualisation
df_group.head()

In [ ]:
# Créer un Dash app
app3 = d.dash.Dash(__name__)

# Créer votre graphique initial
fig = d.px.bar(df_group, x='name', y='Unemployment_rate', color='name',
             labels={'name': 'Pays', 'Unemployment_rate': 'Taux de chômage moyen'},
             template='plotly_dark')

# Créer la mise en page de votre application Dash
app3.layout = d.html.Div([
    
    # Sélecteur de variables
    d.dcc.Dropdown(
        id='variable-dropdown',
        options=[
            {'label': 'Unemployment Rate', 'value': 'Unemployment_rate'},
            {'label': 'GDP growth Rate', 'value': 'GDP_rate'},
            {'label': 'Life expentancy', 'value': 'life_expentancy'},
            {'label': 'Population growth rate', 'value': 'pop_growth_rate'}
          
        ],
        value='Unemployment_rate',  # valeur par défaut de la variable representée
        multi=False,
    ),
    
    # Graphique
    d.dcc.Graph(
        id='bar',
        figure=fig
    )
])

# Définir la logique de la mise à jour du graphique en fonction de la sélection du sélecteur
@app3.callback(
    d.Output('bar', 'figure'),
    [d.Input('variable-dropdown', 'value')]
)
def update_graph3(selected_variable):
    #fig = px.line(filtered_df, x='YEAR', y=["Unemployment_rate"], color='COUNTRY')
    fig = d.px.bar(df_group, x='name', y=[selected_variable], color='name',
                   labels={'name': 'Pays', selected_variable: selected_variable},
                   template='plotly_dark')
    # Afficher le graphique interactif
    
    return fig


# Exécuter l'application Dash
if __name__ == '__main__':
    app3.run_server(debug=True, port=8052)

# <span style="color:green; text-align:center;">III- Analyse descriptive </span>

Pour l'analyse des séries temporelles, une fonction est définie dans le fichier 'declarations.py'. Ladite fonction 'analyse_serie_temporelle_pays' prend en arguments, le jeu de données, l'indicateur (taux de chômage, taux de croissance...) et le pays à considérer.

Cette dernière renvoie, le plot :<br>

<div style="margin-left: 20px;">
-de la série temporelle brute, <br>
-de la série temporelle lissée par une moyenne mobile d'ordre 4,<br> 
-de la décomposition de la série temporelle en composante saisonnière, composante tendancielle et rédisuelle. <br>
    </div>

Enfin, elle permet de tester la stationnarité de la série (Augmented Dickey-Fuller test).<br>

<u>**RAPPEL :**</u>

**Hypothèses du Test de Dickey-Fuller Augmenté (ADF)**

<div style="margin-left: 20px;">

- **$H_0$ :** La série temporelle a une racine unitaire, ce qui indique la non-stationnarité.

- **$H_1$ :** La série temporelle n'a pas de racine unitaire, ce qui indique la stationnarité.
    </div>
**Interprétation des résultats :**

<div style="margin-left: 20px;">

- Si la statistique du test est inférieure à la valeur critique au seuil de signification (par exemple, 0.05), on aurait tendance à rejeter l'hypothèse nulle et conclure que la série temporelle est stationnaire.<br>

- Sinon, on ne peut pas rejeter l'hypothèse nulle, ce qui suggère que la série temporelle est non stationnaire.
    </div>

**NB :** Le test est implémenté par la bibliothèque Statsmodels de Python.


### <span style="color:green; text-align:center;">III-1. Série temporelle : Taux de chômage </span>

In [ ]:
# EXEMPLE : Utilisation de la fonction avec le DataFrame 'final_data', le pays 'AUS', et la variable 'Unemployment_rate'
d.analyse_serie_temporelle_pays(df,'Unemployment_rate', 'AUS')

### <span style="color:green; text-align:center;">III-2. Série temporelle : Taux de croissance du PIB </span>

In [ ]:
# EXEMPLE : Utilisation de la fonction avec le DataFrame 'final_data', le pays 'PHL', et la variable 'GDP_rate'
d.analyse_serie_temporelle_pays(df,'GDP_rate', 'PHL')

### <span style="color:green; text-align:center;">III-3. Série temporelle : Croissance démographique </span>

In [ ]:
# EXEMPLE : Utilisation de la fonction avec le DataFrame 'final_data', le pays 'PHL', et la variable 'GDP_rate'
d.analyse_serie_temporelle_pays(df,'pop_growth_rate', 'USA')

### <span style="color:green; text-align:center;">III-4. Série temporelle : Espérance de vie </span>

In [ ]:
# EXEMPLE : Utilisation de la fonction avec le DataFrame 'final_data', le pays 'PHL', et la variable 'GDP_rate'
d.analyse_serie_temporelle_pays(df,'life_expentancy', 'CAN')

## <span style="color:green; text-align:center;">III-5. APPLICATION 4 : </span>

Dans ce qui suit, la fonction 'analyse_serie_temporelle_pays' sera implémentée dans une application dash.

In [ ]:
df=world_df.copy()

In [ ]:
# Créer un Dash app
app = d.dash.Dash(__name__)

# Liste des pays disponibles dans le dataframe
pays_disponibles = df['name'].unique()

app.layout = d.html.Div([
    d.html.H1("Analyse de Série Temporelle"),
    
    
   # Sélecteur de variables
    d.dcc.Dropdown(
        id='variable-dropdown',
        options=[
            {'label': 'Unemployment Rate', 'value': 'Unemployment_rate'},
            {'label': 'GDP growth Rate', 'value': 'GDP_rate'},
            {'label': 'Life expentancy', 'value': 'life_expentancy'},
            {'label': 'Population growth rate', 'value': 'pop_growth_rate'}
          
        ],
        value='Unemployment_rate',  # valeur par defaut de la variable representée
        multi=False,
    ),
    
    
    # Sélection du pays
    d.html.Label("Choisissez le pays :"),
    d.dcc.Dropdown(
        id='dropdown-pays',
        options=[
            {'label': pays, 'value': pays} for pays in pays_disponibles
        ],
        value=pays_disponibles[0]
    ),
    
    # Graphiques
    d.dcc.Graph(id='time-series-plot'),
    d.dcc.Graph(id='decomposition-plot'),
])

@app.callback(
    [d.Output('time-series-plot', 'figure'),
     d.Output('decomposition-plot', 'figure')],
    [d.Input('variable-dropdown', 'value'),
     d.Input('dropdown-pays', 'value')]
)
def update_graph(indicateur, pays):
    # Sélectionner la série temporelle du pays spécifique
    serie_temporelle = df[indicateur].dropna()
    
    # Moyenne mobile d'ordre 4
    rolling_mean = serie_temporelle.rolling(window=4).mean()

    # Créer le premier graphique (série temporelle et moyenne mobile)
    time_series_fig = d.px.line(x=serie_temporelle.index, y=[serie_temporelle, rolling_mean],
                              labels={'variable': 'Type', 'value': 'Valeur'},
                              title=f'Série Temporelle et Moyenne Mobile - {indicateur} en {pays}',
                              color_discrete_map={'0': 'blue', '1': 'red'})
    
    # Décomposition saisonnière
    decomposition = d.seasonal_decompose(serie_temporelle, model='multiplicative', period=4)

    # Créer le deuxième graphique (décomposition)
    decomposition_fig = d.px.line(x=serie_temporelle.index, y=[decomposition.trend, decomposition.seasonal, decomposition.resid],
                                labels={'variable': 'Composant', 'value': 'Valeur'},
                                title=f'Décomposition - {indicateur} en {pays}',
                                color_discrete_map={'0': 'green', '1': 'orange', '2': 'gray'})

    return time_series_fig, decomposition_fig

if __name__ == '__main__':
    app.run_server(debug=True)